# Training Notebook

# Overview
    - update notebooks to get datasets if on kaggle
## Training Notes
    - Trained on kaggle GPU t4 x2
    - 
    
## Changes and Iterations

## to try

deberta hyperparameter results https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/discussion/483780


# Imports


In [1]:
!pip install seqeval evaluate -q
!pip download  seqeval -q

In [2]:
from pathlib import Path
import os

import json
import argparse
from itertools import chain
from functools import partial

import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
import evaluate
from datasets import Dataset, features
import numpy as np
import pandas as pd

2024-04-08 23:29:12.512818: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 23:29:12.512957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 23:29:12.796598: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Configuration

In [3]:
#DATA_PATH = '../input/pii-detection-removal-from-educational-data'

TRAINING_DATA_PATH = "/kaggle/input/pii-detect-miniset-and-validation-ds/mini_no_overlap.json"
VAL_DATA_PATH = "/kaggle/input/pii-detect-miniset-and-validation-ds/val.json"
TRAINING_MODEL_PATH = "microsoft/deberta-v3-xsmall" #pretrained backbone model
TRAINING_MAX_LENGTH = 1024 # max size of input sequence for training

#Here or in body of notebook?
# TRAIN_BATCH_SIZE = 2 * 8 # size of the input batch in training, x 2 as two GPUs
# EPOCHS = 6 # number of epochs to train
# LR_MODE = "exp" # lr scheduler mode from one of "cos", "step", "exp"

FINE_TUNED_NAME = "deberta3_xsmall_pii2d_1024_mini_v2"
OUTPUT_DIR = "/kaggle/working/"

NOTEBOOK_SEED= 42

LABEL_SET = [
    'B-NAME_STUDENT', 'B-EMAIL','B-USERNAME', 'B-ID_NUM', 'B-PHONE_NUM',
    'B-URL_PERSONAL', 'B-STREET_ADDRESS',
    'I-NAME_STUDENT', 'I-EMAIL','I-USERNAME', 'I-ID_NUM', 'I-PHONE_NUM',
    'I-URL_PERSONAL', 'I-STREET_ADDRESS', 'O'
]

# #print files with pathname
# for dirname, _, filenames in os.walk(DATA_PATH):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [4]:
# Trainer API Configs
# if commented out then it is commented out in trainer API args

#LR = 2e-5  # Initial learning rate
GRADIENT_ACCUMULATION_STEPS = 2  # How many batches to accumulate gradient before optimization if batch size limited by GPU memory
REPORT_TO = "none"  # Where training report progress, "none" prevents wandb login
NUM_TRAIN_EPOCHS = 2  # Number of training epochs
PER_DEVICE_TRAIN_BATCH_SIZE = 4  # Batch size based per GPU
DO_EVAL = True  # Whether or not to perform eval during training
EVALUATION_STRATEGY = "epoch"  # When to evaluate during training {no, steps or epoch}
LOGGING_DIR = OUTPUT_DIR + "/logs"  # Directory to save training logs
#LOGGING_STEPS = 100  # Log training progress every X steps
LOAD_BEST_MODEL_AT_END = True  # Load the best model at the end of training
METRIC_FOR_BEST_MODEL = "f1"  # Metric to determine the best model ("accuracy", f1...)
GREATER_IS_BETTER = True  # If higher eval metric is better. True for f1 and acc
SAVE_TOTAL_LIMIT = 1  # How many checkpoints to keep at end (1 means most recent)
# WARMUP_RATIO = 0.1  # Steps to gradually increase learning rate. Can help stabilize training at beginning
# WEIGHT_DECAY = 0.01  # L2 regularization to prevent overfitting

In [5]:
# #TEMP TEST
# from kaggle_secrets import UserSecretsClient
# import wandb

# user_secrets = UserSecretsClient()
# wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
# wandb.login(key=wandb_api_key)

## Hardware Selection?

## Data Selection

In [6]:
#data from orginal training json
data = json.load(open(TRAINING_DATA_PATH))
org_data_df = pd.DataFrame(data)
train_df = org_data_df

print("Training Data: ", len(data))

Training Data:  1837


# Tokenize Data

In [7]:
#prep data for NER training by tokenize the text and align labels to tokens
def tokenize(example, tokenizer, label2id, max_length):
    """This function ensures that the text is correctly tokenized and the labels 
    are correctly aligned with the tokens for NER training.

    Args:
        example (dict): The example containing the text and labels.
        tokenizer (Tokenizer): The tokenizer used to tokenize the text.
        label2id (dict): A dictionary mapping labels to their corresponding ids.
        max_length (int): The maximum length of the tokenized text.

    Returns:
        dict: The tokenized example with aligned labels.

    Reference: credit to https://www.kaggle.com/code/valentinwerner/915-deberta3base-training/notebook
    """

    # rebuild text from tokens
    text = []
    labels = []

    #iterate through tokens, labels, and trailing whitespace using zip to create tuple from three lists
    for t, l, ws in zip(
        example["tokens"], example["labels"], example["trailing_whitespace"]
    ):
        text.append(t)
        
        #extend so we can add multiple elements to end of list if ws
        labels.extend([l] * len(t))
        if ws:
            text.append(" ")
            labels.append("O")

    #Tokenize text and return offsets for start and end character position. Limit length of tokenized text.
    tokenized = tokenizer("".join(text), return_offsets_mapping=True, max_length=max_length, truncation=True)

    #convert to np array for indexing
    labels = np.array(labels)

    # join text list into a single string 
    text = "".join(text)
    token_labels = []

    #iterate through each tolken
    for start_idx, end_idx in tokenized.offset_mapping:
        #if special tolken (CLS token) then append O
        #CLS : classification token added to the start of each sequence
        if start_idx == 0 and end_idx == 0:
            token_labels.append(label2id["O"])
            continue

        # case when token starts with whitespace
        if text[start_idx].isspace():
            start_idx += 1

        #append orginal label to token_labels
        token_labels.append(label2id[labels[start_idx]])

    length = len(tokenized.input_ids)

    return {**tokenized, "labels": token_labels, "length": length}

In [8]:
#Set up labeling for NER with #Targets: B-Beginning entity, I-inside entity, O- outside entity

#target labels identified in the training data- changed to all possible target labels
target = [
    'B-NAME_STUDENT', 'B-EMAIL','B-USERNAME', 'B-ID_NUM', 'B-PHONE_NUM',
    'B-URL_PERSONAL', 'B-STREET_ADDRESS',
    'I-NAME_STUDENT', 'I-EMAIL','I-USERNAME', 'I-ID_NUM', 'I-PHONE_NUM',
    'I-URL_PERSONAL', 'I-STREET_ADDRESS', 'O'
]

# NOTE: changed to all possible labels since training data is missing some
# #Extract all unique labels w/ list comprehension. Use chain to flatten list of lists
# all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))

all_labels= target

#Create dictionary of label to id
label2id = {l: i for i,l in enumerate(all_labels)}

#Create dictionary of id to label
id2label = {v:k for k,v in label2id.items()}

print(id2label)

{0: 'B-NAME_STUDENT', 1: 'B-EMAIL', 2: 'B-USERNAME', 3: 'B-ID_NUM', 4: 'B-PHONE_NUM', 5: 'B-URL_PERSONAL', 6: 'B-STREET_ADDRESS', 7: 'I-NAME_STUDENT', 8: 'I-EMAIL', 9: 'I-USERNAME', 10: 'I-ID_NUM', 11: 'I-PHONE_NUM', 12: 'I-URL_PERSONAL', 13: 'I-STREET_ADDRESS', 14: 'O'}


In [9]:
#load tokenizer based on pretrained model
tokenizer = AutoTokenizer.from_pretrained(TRAINING_MODEL_PATH)

#convert to hugging face Dataset object
ds = Dataset.from_pandas(train_df)

# Map the tokenize function to your dataset
ds = ds.map(
    tokenize,
    fn_kwargs={      # pass keyword args
        "tokenizer": tokenizer,
        "label2id": label2id,
        "max_length": TRAINING_MAX_LENGTH
    }, 
    num_proc=2   #apply in paralell using 2 processes
)
#TODO
#add class encoding toreplaces categorical class labels with numerical values, making it easier for machine learning models?
#ds = ds.class_encode_column("group")

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


#0:   0%|          | 0/919 [00:00<?, ?ex/s]

#1:   0%|          | 0/918 [00:00<?, ?ex/s]

In [10]:
#Validation dataset
val_df = pd.DataFrame(json.load(open(VAL_DATA_PATH)))
#convert to hugging face Dataset object
val_ds = Dataset.from_pandas(val_df)

# Map the tokenize function to your dataset
val_ds = val_ds.map(
    tokenize,
    fn_kwargs={      # pass keyword args
        "tokenizer": tokenizer,
        "label2id": label2id,
        "max_length": TRAINING_MAX_LENGTH
    }, 
    num_proc=2   #apply in paralell using 2 processes
)

#0:   0%|          | 0/344 [00:00<?, ?ex/s]

#1:   0%|          | 0/343 [00:00<?, ?ex/s]

# Metrics and Training

In [11]:
#TODO- Review and confirm works
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score


def compute_metrics(p, all_labels):
    """Compute the F1, recall, precision metrics for a NER task.

    Args:
        p (Tuple[np.ndarray, np.ndarray]): The predictions and labels.
        all_labels (List[str]): The list of all possible labels.

    Returns:
        Dict[str, float]: The computed metrics (recall, precision, f1_score).
    Ref: https://www.kaggle.com/code/valentinwerner/915-deberta3base-training/notebook
    """
    #Note: seqeval framework for sequence labeling like NER
    
    # Unpack the predictions and labels
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f5_score = (1 + 5*5) * recall * precision / (5*5*precision + recall + 1e-10)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f5': f5_score
    }
    return results

In [12]:
#load
model = AutoModelForTokenClassification.from_pretrained(
    TRAINING_MODEL_PATH,        #pretrained model
    num_labels=len(all_labels), #num of unique labels for finetuning
    id2label=id2label,          #dicts for converting in fine tuning
    label2id=label2id,
    ignore_mismatched_sizes=True #pretrained model might have been trained on different num of labels
)

#collate list of sample from dataset into batches. 16 might be benefical for GPU architecture
collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# #TODO Freezing Layers in DeBERTa V3 Models
# FREEZE_EMBEDDINGS = False
# FREEZE_LAYERS = 6

# if FREEZE_EMBEDDINGS:
#     print('Freezing embeddings.')
#     for param in model.deberta.embeddings.parameters():
#         param.requires_grad = False
        
# if FREEZE_LAYERS>0:
#     print(f'Freezing {FREEZE_LAYERS} layers.')
#     for layer in model.deberta.encoder.layer[:FREEZE_LAYERS]:
#         for param in layer.parameters():
#             param.requires_grad = False

In [14]:
#Configure training process
#no validation set specified
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,  # Directory to save checkpoints and logs
    fp16=True,  # mix-precision training on 16 bit to reduce memory and speed up training
    learning_rate=1e-5,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    report_to="none",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    overwrite_output_dir=True,
    load_best_model_at_end=True,
    lr_scheduler_type='cosine',
    metric_for_best_model="f5",
    greater_is_better=True,
    weight_decay=0.01
)

#inialize trainer for training and evaluation interface
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=ds,
    eval_dataset= val_ds,
    data_collator=collator, 
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics, all_labels=all_labels), #partial to fix all_label argument
)

In [15]:
%%time

#train model 
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Recall,Precision,F5
1,No log,0.009569,0.000000,0.000000,0.000000
2,0.189500,0.007368,0.000000,0.000000,0.000000


/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


CPU times: user 7min 27s, sys: 1min, total: 8min 28s
Wall time: 6min 31s


TrainOutput(global_step=920, training_loss=0.10608279601387356, metrics={'train_runtime': 390.5843, 'train_samples_per_second': 9.406, 'train_steps_per_second': 2.355, 'total_flos': 424458696471264.0, 'train_loss': 0.10608279601387356, 'epoch': 2.0})

In [16]:
trainer.save_model(FINE_TUNED_NAME)
tokenizer.save_pretrained(FINE_TUNED_NAME)

('deberta3_xsmall_pii2d_1024_mini_v2/tokenizer_config.json',
 'deberta3_xsmall_pii2d_1024_mini_v2/special_tokens_map.json',
 'deberta3_xsmall_pii2d_1024_mini_v2/spm.model',
 'deberta3_xsmall_pii2d_1024_mini_v2/added_tokens.json',
 'deberta3_xsmall_pii2d_1024_mini_v2/tokenizer.json')